# Keras MNIST mlp

Dimension: 784x10<br>
Training size: 60,000<br>
Test size: 10,000

In [1]:
import keras as ks

Using Theano backend.


In [2]:
import numpy as np
from functools import reduce
import datetime

### Load data

In [3]:
from mnist import MNIST
#you can find python-mnist source code on https://github.com/sorki/python-mnist

datahandler = MNIST('../mnist_data_loader') #change for data path
train_data = datahandler.load_training()
test_data = datahandler.load_testing()

In [4]:
train_image_array = np.asarray(train_data[0])
test_image_array = np.asarray(test_data[0])

In [5]:
train_label_list = []
for i in train_data[1]:
    l = [0]*10
    l[i] = 1
    train_label_list.append(l)

train_label_array = np.asarray(train_label_list)

In [6]:
test_label_list = []
for i in test_data[1]:
    l = [0]*10
    l[i] = 1
    test_label_list.append(l)

test_label_array = np.asarray(test_label_list)

### Create and train model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.optimizers import SGD

In [8]:
model = Sequential([
    Dense(10, batch_input_shape=(None, 784), activation = 'sigmoid'),
    Dense(10, activation = 'softmax'),
])

In [9]:
sgd = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics = ['accuracy'])

In [10]:
t = datetime.datetime.now()
hist = model.fit(train_image_array, 
                 train_label_array,
                 nb_epoch=10, 
                 batch_size=10)
print("Tiempo de ejecución: {}".format(datetime.datetime.now()-t))

Epoch 1/10
60000/60000 [==============================] - 3s - loss: 0.2622 - acc: 0.9012     
Epoch 2/10
60000/60000 [==============================] - 3s - loss: 0.2052 - acc: 0.9103     
Epoch 3/10
60000/60000 [==============================] - 3s - loss: 0.1716 - acc: 0.9331     
Epoch 4/10
60000/60000 [==============================] - 3s - loss: 0.1487 - acc: 0.9520     
Epoch 5/10
60000/60000 [==============================] - 3s - loss: 0.1335 - acc: 0.9573     
Epoch 6/10
60000/60000 [==============================] - 3s - loss: 0.1229 - acc: 0.9616     
Epoch 7/10
60000/60000 [==============================] - 3s - loss: 0.1144 - acc: 0.9657     
Epoch 8/10
60000/60000 [==============================] - 4s - loss: 0.1082 - acc: 0.9673     
Epoch 9/10
60000/60000 [==============================] - 4s - loss: 0.1039 - acc: 0.9682     
Epoch 10/10
60000/60000 [==============================] - 4s - loss: 0.1014 - acc: 0.9688     
Tiempo de ejecución: 0:00:38.723407


### Make predictions

In [11]:
p = model.predict([test_image_array])

In [12]:
print("Dimensión predicción: {0}".format(p.shape)) #check prediction dimensions

Dimensión predicción: (10000, 10)


# Metrics

### Confusion matrix
Useful for metric calculation

In [13]:
conf_mtx = np.zeros([10, 10])
for j in range(0, len(p)):
    prediction = p[j]
    max_pred = reduce(lambda x, y: (x if prediction[x] > prediction[y] else y), range(0,10))
    actual_label = list(filter(lambda i: test_label_array[j][i], range(0,10)))[0]
    conf_mtx[actual_label][max_pred] += 1
np.set_printoptions(suppress=True)
print(conf_mtx)

[[  918.     0.     8.     5.     1.    12.    17.     1.    14.     4.]
 [    0.  1091.    10.     4.     2.     0.     3.     2.    23.     0.]
 [   17.    13.   928.     3.    11.     4.    16.    13.    22.     5.]
 [   33.     3.    87.   769.     9.    41.     4.    19.    15.    30.]
 [    2.     3.    14.     1.   911.     1.     9.     2.     7.    32.]
 [   48.     9.     7.    40.    22.   539.    33.    19.   160.    15.]
 [   15.     3.    24.     1.    30.     8.   850.     0.    26.     1.]
 [    2.    10.    36.     2.    14.     2.     1.   877.     8.    76.]
 [   12.     9.    29.    18.    32.     8.    16.    19.   786.    45.]
 [   11.     2.     2.    10.    71.     8.     1.    26.    11.   867.]]


### Accuracy

In [14]:
acc = (sum(conf_mtx[i][i] for i in range(0, 10))/10000)

In [15]:
print('Accuracy: {:.5f}'.format(acc))

Accuracy: 0.85360


### Recall

In [16]:
recall = lambda i: (conf_mtx[i][i]/sum(conf_mtx[i][j] for j in range(0,10)))

In [17]:
recall_sum = 0
for i in range(0,10):
    rcl = recall(i)
    recall_sum += rcl
    print('Recall {}: {:.5f}'.format(i, rcl))
print()
print('Recall mean: {:.5f}'.format(recall_sum/10))

Recall 0: 0.93673
Recall 1: 0.96123
Recall 2: 0.89922
Recall 3: 0.76139
Recall 4: 0.92770
Recall 5: 0.60426
Recall 6: 0.88727
Recall 7: 0.85311
Recall 8: 0.80698
Recall 9: 0.85927

Recall mean: 0.84972


### Precision

In [18]:
precision = lambda i: (conf_mtx[i][i]/sum(conf_mtx[j][i] for j in range(0,10)))

In [19]:
precision_sum = 0
for i in range(0,10):
    label_precision = precision(i)
    precision_sum += label_precision
    print('Precision {}: {:.5f}'.format(i, label_precision))
print()
print('Precision mean: {:.5f}'.format(precision_sum/10))

Precision 0: 0.86767
Precision 1: 0.95451
Precision 2: 0.81048
Precision 3: 0.90152
Precision 4: 0.82593
Precision 5: 0.86517
Precision 6: 0.89474
Precision 7: 0.89673
Precision 8: 0.73321
Precision 9: 0.80651

Precision mean: 0.85565
